In [1]:
import matplotlib.pyplot as plt
import astropy
import os
import pandas as pd
import numpy as np
import xlsxwriter

from astropy.io import fits
from astropy.wcs import WCS
import numpy as np
import os
import tracemalloc
import pandas as pd
import matplotlib as mpl
import io
from skimage.transform import resize
import cv2
from astronomaly.base.base_dataset import Dataset
from astronomaly.base import logging_tools
import matplotlib.pyplot as plt  # noqa: E402

In [2]:
catalogue_file = '/home/verlon/Desktop/Astronomaly/Data/Input/Coadd_0260/0260m062/Catalogue/tractor-0260m062.fits'

In [3]:
image_file_g = '/home/verlon/Desktop/Astronomaly/Data/Input/Coadd_0260/0260m062/Images/g-legacysurvey-0260m062-image.fits.fz'
image_file_r = '/home/verlon/Desktop/Astronomaly/Data/Input/Coadd_0260/0260m062/Images/r-legacysurvey-0260m062-image.fits.fz'
image_file_z = '/home/verlon/Desktop/Astronomaly/Data/Input/Coadd_0260/0260m062/Images/z-legacysurvey-0260m062-image.fits.fz'

In [4]:
class AstroImage:
    def __init__(self, filenames, file_type='fits', fits_index=0, name=''):
        """
        Lightweight wrapper for an astronomy image from a fits file

        Parameters
        ----------
        filenames : list of files
            Filename of fits file to be read. Can be length one if there's only
            one file or multiple if there are multiband images
        fits_index : integer
            Which HDU object in the list to work with

        """
        print('Reading image data from %s...' % filenames[0])
        self.filenames = filenames
        self.file_type = file_type
        self.metadata = {}
        self.wcs = None
        self.fits_index = fits_index
        self.hdul_list = []

        try:
            for f in filenames:     
                hdul = fits.open(f, memmap=True)
                self.hdul_list.append(hdul)               

        except FileNotFoundError:
            raise FileNotFoundError("File", f, "not found")

        # get a test sample
        self.get_image_data(0, 10, 0, 10)

        if len(name) == 0:
            self.name = self._strip_filename()
        else:
            self.name = name

        print('Done!')

    def get_image_data(self, row_start, row_end, col_start, col_end):
        """Returns the image data from a fits HDUlist object

        Parameters
        ----------
        Returns
        -------
        np.array
            Image data
        """
        images = []
        rs = row_start
        re = row_end
        cs = col_start
        ce = col_end

        for hdul in self.hdul_list:
            if self.fits_index is None:
                for i in range(len(hdul)):
                    self.fits_index = i
                    # snap1 = tracemalloc.take_snapshot()
                    dat = hdul[self.fits_index].data
                    # snap2 = tracemalloc.take_snapshot()
                    # diff = snap2.compare_to(snap1, 'lineno')
                    # print(diff[0].size_diff)
                    if dat is not None:
                        if len(dat.shape) > 2:
                            dat = dat[0][0]
                        image = dat[rs:re, cs:ce]
                        break
                self.metadata = dict(hdul[self.fits_index].header)
                if self.wcs is None:
                    self.wcs = WCS(hdul[self.fits_index].header, naxis=2)
            else:
                dat = hdul[self.fits_index].data
                if len(dat.shape) > 2:
                    dat = dat[0][0]
                image = dat[rs:re, cs:ce]

            if len(image.shape) > 2:
                image = np.squeeze(image)
            images.append(image)

        if len(images) > 1:
            # Should now be a 3d array with multiple channels
            image = np.dstack(images)
            self.metadata['NAXIS3'] = image.shape[-1]
        else:
            image = images[0]  # Was just the one image
        return image

    def clean_up(self):
        """
        Closes all open fits files so they don't remain in memory.
        """
        print("Closing Fits files...")
        for hdul in self.hdul_list:
            hdul.close()
        logging_tools.log("Fits files closed successfully.")
        print("Files closed.")

    def _strip_filename(self):

        """
        Tiny utility function to make a nice formatted version of the image 
        name from the input filename string

        Returns
        -------
        string
            Formatted file name

        """
        s1 = self.filenames[0].split(os.path.sep)[-1]
        # extension = s1.split('.')[-1]
        return s1

    def get_coords(self, x, y):
        """
        Returns the RA and DEC coordinates for a given set of pixels.

        Parameters
        ----------
        x : int
            x pixel value
        y : y
            y pixel value

        Returns
        -------
        ra, dec
            Sky coordinates
        """

        return self.wcs.wcs_pix2world(x, y, 0)


In [5]:
filenames = image_file_g
filenames

'/home/verlon/Desktop/Astronomaly/Data/Input/Coadd_0260/0260m062/Images/g-legacysurvey-0260m062-image.fits.fz'

In [6]:
hdul_list = []


hdul = fits.open(filenames, memmap=True)
AstroImage.__init__.hdul_list.append(hdul)

AttributeError: 'function' object has no attribute 'hdul_list'

In [7]:
class ImageCycler:
    def __init__(self, images, xlabels=None):
        """
        Convenience object to cycle through a list of images inside a jupyter 
        notebook.

        Parameters
        ----------
        images : list
            List of numpy arrays to display as images
        xlabels : list, optional
            List of custom labels for the images
        """

        self.current_ind = 0
        self.images = images
        self.xlabels = xlabels

    def onkeypress(self, event):
        """
        Matplotlib event handler for left and right arrows to cycle through 
        images.

        Parameters
        ----------
        event

        Returns
        -------

        """
        plt.gcf()
        if event.key == 'right' and self.current_ind < len(self.images):
            self.current_ind += 1

        elif event.key == 'left' and self.current_ind > 0:
            self.current_ind -= 1

        plt.clf()
        event.canvas.figure.gca().imshow(
            self.images[self.current_ind], origin='lower', cmap='hot')

        if self.xlabels is not None:
            plt.xlabel(self.xlabels[self.current_ind])
        plt.title(self.current_ind)
        event.canvas.draw()

    def cycle(self):
        """
        Creates the plots and binds the event handler
        """

        fig = plt.figure()
        fig.canvas.mpl_connect('key_press_event', self.onkeypress)
        plt.imshow(self.images[self.current_ind], origin='lower', cmap='hot')
        plt.title(self.current_ind)

In [8]:
catalogue_file = '/home/verlon/Desktop/Astronomaly/Data/Input/Coadd_0260/0260m062/Catalogue/tractor-0260m062.fits'

In [9]:
image_file_g = '/home/verlon/Desktop/Astronomaly/Data/Input/Coadd_0260/0260m062/Images/g-legacysurvey-0260m062-image.fits.fz'
image_file_r = '/home/verlon/Desktop/Astronomaly/Data/Input/Coadd_0260/0260m062/Images/r-legacysurvey-0260m062-image.fits.fz'
image_file_z = '/home/verlon/Desktop/Astronomaly/Data/Input/Coadd_0260/0260m062/Images/z-legacysurvey-0260m062-image.fits.fz'

In [10]:
def convert_tractor_catalogue(catalogue_file, image_file, image_name=''):
    """
    Converts a tractor fits file to a pandas dataframe to be given
    directly to an ImageDataset object.

    Parameters
    ----------
    catalogue_files : string
        tractor catalogue in fits table format 
    image_file:
        The image corresponding to this catalogue (to extract pixel information
        and naming information)
    """

    catalogue = astropy.table.Table(astropy.io.fits.getdata(catalogue_file,ignore_missing_end=True))

    dataframe = {}
    for name in catalogue.colnames:
        data = catalogue[name].tolist()
        dataframe[name] = data
    
    old_catalogue = pd.DataFrame(dataframe)
    hdul = astropy.io.fits.open(image_file)

    if len(image_name) == 0:
        original_image = image_file.split(os.path.sep)[-1]
    else:
        original_image = image_name
    
    #w = astropy.wcs.WCS(hdul[0].header, naxis=2)
    #x, y = w.wcs_world2pix(old_catalogue['ra'], old_catalogue['dec'], 1)
    
    new_catalogue = pd.DataFrame()
    new_catalogue['objid'] = old_catalogue['objid']
    new_catalogue['original_image'] = [original_image] * len(new_catalogue)
    new_catalogue['flux_g'] = old_catalogue['flux_g']
    new_catalogue['flux_r'] = old_catalogue['flux_r']
    new_catalogue['flux_z'] = old_catalogue['flux_z']
    new_catalogue['x'] = old_catalogue['bx'].astype('int')
    new_catalogue['y'] = old_catalogue['by'].astype('int')
    new_catalogue['ra'] = old_catalogue['ra']
    new_catalogue['dec'] = old_catalogue['dec']
    
    new_catalogue['type'] = old_catalogue['type']
    
    return new_catalogue

In [11]:
new_catalogue = convert_tractor_catalogue(catalogue_file, image_file_g, image_name='')

AttributeError: module 'astropy' has no attribute 'table'

In [12]:
if new_catalogue is not None:
    if 'objid' in new_catalogue.columns:
        new_catalogue.set_index('objid')
    metadata = new_catalogue
#else:
#    inds = []
#    file_paths = []
#    for f in self.files:
#        extension = f.split('.')[-1]
#        if extension in self.known_file_types:
#            inds.append(
#                f.split(os.path.sep)[-1][:-(len(extension) + 1)])
#            file_paths.append(f)
#    self.metadata = pd.DataFrame(index=inds, 
#                                    data={'filename': file_paths})

index = metadata.index.values

NameError: name 'new_catalogue' is not defined

In [13]:
new_catalogue.columns

NameError: name 'new_catalogue' is not defined

In [14]:
cols = ['original_image', 'x', 'y']

In [ ]:
cols

In [ ]:
if 'ra' in new_catalogue.columns:
    cols.append('ra')
if 'dec' in new_catalogue.columns:
    cols.append('dec')
if 'peak_flux' in new_catalogue.columns:
    cols.append('peak_flux')

In [ ]:
cols

In [ ]:
met = {}

for c in cols:
    met[c] = new_catalogue[c].values

In [ ]:
the_index = np.array(new_catalogue['objid'].values)#, dtype='str')

In [ ]:
the_index

In [ ]:
metadata = pd.DataFrame(met, index=the_index)

In [ ]:
metadata['x'] = metadata['x'].astype('int')

In [ ]:
metadata['y'] = metadata['y'].astype('int')

In [ ]:
metadata

In [ ]:
window_size=128
window_shift = window_size

In [ ]:
idx = 1

In [ ]:
image = fits.open(image_file_g)
image

In [ ]:
x0 = metadata.loc[idx, 'x']

In [ ]:
y0 = metadata.loc[idx, 'y']

In [ ]:
original_image = metadata.loc[idx, 'original_image']
original_image

In [ ]:
this_image = images[original_image]

In [ ]:
filenames = image_file_g

In [ ]:
AstroImage( filenames, file_type='fits', fits_index=0, name='')

In [ ]:
original_image

In [ ]:
            xvals = np.arange(window_size // 2, 
                              img.shape[1] - window_size // 2, 
                              window_shift)
            yvals = np.arange(window_size // 2, 
                              img.shape[0] - window_size // 2,
                              window_shift)
            X, Y = np.meshgrid(xvals, yvals)

In [ ]:
new_catalogue

In [ ]:
catalogue_no_PSFs = new_catalogue[new_catalogue.type.astype("S") != b'PSF ']

In [ ]:
catalogue_no_PSFs.sort_values('flux_g', ascending=False)

In [ ]:
range(len(catalogue_no_PSFs))

In [ ]:
ra_value = catalogue_no_PSFs.sort_values('flux_g', ascending=False)['ra'][0]
dec_value = catalogue_no_PSFs.sort_values('flux_g', ascending=False)['dec'][0]

In [ ]:
ra_value

In [ ]:
dec_value

In [ ]:
ra = catalogue_no_PSFs.sort_values('flux_g', ascending=False)['ra'].reset_index()
ra

In [ ]:
import webbrowser  
import time

for i in range(len(catalogue_no_PSFs)):
    ra = catalogue_no_PSFs.sort_values('flux_g', ascending=False)['ra'].reset_index()
    dec = catalogue_no_PSFs.sort_values('flux_g', ascending=False)['dec'].reset_index()
    
    ra_value = ra.loc[i].ra
    dec_value = dec.loc[i].dec
    
    webbrowser.open(
        'http://legacysurvey.org/viewer/fits-cutout/?ra=' +str(ra_value) + '&dec=' +str(dec_value) + '&layer=dr8&pixscale=0.27&bands=grz',
        new=0,
        autoraise=True
    )
    
    time.sleep(1)

In [ ]:
from astropy import io
from astropy.io import fits
from astropy import table
from astropy import wcs
import astropy



img = fits.open('/home/verlon/Downloads/DATA/cutout_26.1266_-6.3541.fits')[0]


plt.imshow(img.data[0], cmap='gray')


In [ ]:
img[0]

In [ ]:
plt.imshow(img[1], cmap='gray')